## Partie 1 : Apprentissage des paramètres d’un modèle à partir de données

Dans notre modèle, nous allons considérer qu’à chaque temps :
- chaque individu sain peut rester sain ou devenir infecté
- chaque individu infecté peut rester infecté ou devenir guéri
- chaque individu guéri reste guéri

et que la probabilité de passer d’un état à l’autre **ne dépend que de l’état précédent**.


Suivi d'un individu sur 10 jours : 
$S, S, S, I, I ,I ,I , I, I, R$

La matrice de probabilité de transitions

|  |S | I | R |
| :--|:--:| :--:|--:|
|S   | 2/3 |1/3  |0|
|I   | 1/6 | 5/6 |0|
|R   |0    |  0  |1|

In [1]:
import numpy as np


Nous avons ensuite suivit une population de $5000$ individus, pendant $200$ jours. Pour lire les données
utiliser `np.loadtxt(‘data_exo1.txt’)`. 
Les individus sains sont note 0, les infecté 1 et les guéris 2.

In [2]:
data = np.loadtxt("data_ex1.txt", dtype =int) # Lecture des données

# 0 --> Sain (S)
# 1 --> Infecté (I)
# 2 --> Guéri (R)

def probas(data):
    m = np.zeros((3,3), dtype= float) #Création d'une matrice initialisée à 0
    prec = 9 #Declaration d'une variable précédent


    for individu in data : 
        for state in individu :
            if prec == 9 : #Si c'est la premiere valeur pour un individu on passe au suivant et on le met dans le precedent
                prec = state
                continue
            else : # Sinon on compte la transition dans la matrice m (precedent -> state)
                m[prec][state]+=1
            prec = state
        prec = 9 #A la fin de chaque individu il n'y a plus de precedent

    sum = np.sum(m,axis=1) #On somme toute les lignes
    
    for i in range(3) :
        for j in range(3) :
            m[i][j] = m[i][j]/sum[i] #On chaque case par le nombre total de transition dans une ligne
    
    return m
            


matrice = probas(data)


## Partie 2 : Description du premier modèle

Grâce au graph de transition du document 1 on obtient la matrice de transition A suivante :
|  |S | I | R |
| :--|:--:| :--:|--:|
|S   |0.92 | 0.08 |  0 |
|I   | 0   | 0.93 |0.07|
|R   |  0  |  0   |  1 |

In [6]:
A = [[0.92,0.08,0],[0,0.93,0.07],[0,0,1]]
print(A)

[[0.92, 0.08, 0], [0, 0.93, 0.07], [0, 0, 1]]


In [7]:
def stochastique(matrice) :
    return np.all(matrice >= 0 ) and np.all(np.sum(matrice, axis=1)) == 1

print(stochastique(matrice))

True


Au temps $t = 0$, un individu a une probabilité de 0.9 d’être sain et 0.1 d’être infecté.
Soit $\pi_0$ la distribution de la probabilité initiale.
On a donc $\pi_0 = [0.9;0.1;0]$

In [5]:
pi0 = [0.9,0.1,0]
print(pi0)

[0.9, 0.1, 0]


Distribution :

Au temps $t = 1$ on a donc $\pi_0 \times A$ donc $\pi_1 = [0.828;0.165;0.007]$

In [9]:
pi1 = np.dot(pi0,A)
print(pi1)

[0.828 0.165 0.007]


Au temps $t = 2$ on a donc $\pi_1 \times A$ donc $\pi_2 = [0,76176;0,21969;0,01855]$

In [11]:
pi2 = np.dot(pi1,A)
print(pi2)

[0.76176 0.21969 0.01855]
